### Preprocessing data


In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

In [20]:
ratings=pd.read_csv('C:\\Users\\rsarm\\Desktop\\Fall 2018\\646\\Project\\Data\\goodbooks-10k-master\\goodbooks-10k-master\\ratings.csv')
books=pd.read_csv('C:\\Users\\rsarm\\Desktop\\Fall 2018\\646\\Project\\Data\\goodbooks-10k-master\\goodbooks-10k-master\\books.csv')
print(ratings.columns)
#books['elo']=1200.0
print(books.columns)
print(ratings.head(1))

Index(['user_id', 'book_id', 'rating'], dtype='object')
Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')
   user_id  book_id  rating
0        1      258       5


### Get books liked by a particular User ID

In [30]:

userBooks = ratings.loc[ratings['user_id'] == 1]
userbookdetails = pd.merge(userBooks, books, left_on = 'book_id', right_on = 'goodreads_book_id')
print(userbookdetails)

    user_id  book_id_x  rating  book_id_y  goodreads_book_id  best_book_id  \
0         1         11       5         54                 11        386162   
1         1         33       4        189                 33            33   
2         1         10       4       3753                 10            10   
3         1         36       4       4229                 36            36   
4         1        119       3       3230                119           119   
5         1         13       4        337                 13            13   
6         1       2002       5       9048               2002          2002   
7         1         67       3       3504                 67            67   
8         1        378       3        561                378           378   
9         1         98       3       7683                 98            98   
10        1       5191       4       1311               5191          5191   
11        1        231       3       4081                231    

### Sparse Matrix representation

In [4]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


from scipy.sparse import coo_matrix, csr_matrix
def read_data(filename):
    """ Reads in the last.fm dataset, and returns a tuple of a pandas dataframe
    and a sparse matrix of song/user/playcount """
    # read in triples of user/song/playcount from the input dataset
    data = pd.read_csv(filename,
                             usecols=[0,1,2],        #[36, 11, 10] vrk_pat_primkey,prd_atc_primkey,vdp_aantal
                             names=['song', 'user','plays'],skiprows=1) #[:1000000]   # user = patient, or prescriptionnr song=atc

    data=data.dropna(axis=0, how='any')  #drop nan
    data['plays']=data['plays']+1
    print(data.head())
    # map each song and user to a unique numeric value
    data['user'] = data['user'].astype("category")
    data['song'] = data['song'].astype("category")

    # create a sparse matrix of all the users/plays
    plays = coo_matrix((data['plays'].astype(float),
                       (data['song'].cat.codes.copy(),
                        data['user'].cat.codes.copy())))
    data['song_nr']=data['song'].cat.codes.copy()
    return data, plays,data.groupby(['song_nr','song']).plays.sum(),data['user'].cat.codes.copy()

data,matrix,songsd,user=read_data('C:\\Users\\rsarm\\Desktop\\Fall 2018\\646\\Project\\Data\\goodbooks-10k\\ratings.csv')
data.head()

   song  user  plays
0     1   314      6
1     1   439      4
2     1   588      6
3     1  1169      5
4     1  1185      5


,song,user,plays,song_nr
0,1,314,6,0
1,1,439,4,0
2,1,588,6,0
3,1,1169,5,0
4,1,1185,5,0


### Normalize

In [6]:
from sklearn.preprocessing import normalize


def cosine(plays):
    normalized = normalize(plays)
    return normalized.dot(normalized.T)


def bhattacharya(plays):
    plays.data = np.sqrt(plays.data)
    return cosine(plays)


def ochiai(plays):
    plays = csr_matrix(plays)
    plays.data = np.ones(len(plays.data))
    return cosine(plays)


def bm25_weight(data, K1=1.2, B=0.8):
    """ Weighs each row of the matrix data by BM25 weighting """
    # calculate idf per term (user)
    N = float(data.shape[0])
    idf = np.log(N / (1 + np.bincount(data.col)))

    # calculate length_norm per document (artist)
    row_sums = np.squeeze(np.asarray(data.sum(1)))
    average_length = row_sums.sum() / N
    length_norm = (1.0 - B) + B * row_sums / average_length

    # weight matrix rows by bm25
    ret = coo_matrix(data)
    ret.data = ret.data * (K1 + 1.0) / (K1 * length_norm[ret.row] + ret.data) * idf[ret.col]
    return ret


def bm25(plays):
    plays = bm25_weight(plays)
    return plays.dot(plays.T)

def get_largest(row, N=10):
    if N >= row.nnz:
        best = zip(row.data, row.indices)
    else:
        ind = np.argpartition(row.data, -N)[-N:]
        best = zip(row.data[ind], row.indices[ind])
    return sorted(best, reverse=True)


def calculate_similar_artists(similarity, artists, artistid):
    neighbours = similarity[artistid]
    top = get_largest(neighbours)
    return [(artists[other], score, i) for i, (score, other) in enumerate(top)]


songsd = dict(enumerate(data['song'].cat.categories))
user_count = data.groupby('song').size()
to_generate = sorted(list(songsd), key=lambda x: -user_count[x])

similarity = bm25(matrix)

### SVD

In [7]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
Xr=svd.fit_transform(bm25(matrix))  
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

[0.07071025 0.0289823  0.03094033 0.02356602 0.01866404 0.01791777
 0.01506202 0.01279004 0.01155299 0.00986874 0.00914476 0.00825633
 0.00725814 0.00704479 0.00667714 0.00610331 0.00605892 0.00582119
 0.00564076 0.00558224 0.0051532  0.00489197 0.00458071 0.00438815
 0.004383   0.00417003 0.00410773 0.00403565 0.0039309  0.00383504
 0.003664   0.00363734 0.00354132 0.00343039 0.00339091 0.0033702
 0.00323359 0.00318197 0.00313176 0.00306081 0.00299382 0.00295211
 0.00281956 0.00274961 0.00268104 0.00265187 0.00248863 0.00248133
 0.0023696  0.00233401]
0.41128233580847945


### Cosine similarity

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
Udf=pd.DataFrame(cosine_similarity(Xr))


In [9]:
booknr=4536
print(Udf[booknr].sort_values(ascending=False)[:10])
books[books['id'].isin( Udf[booknr].sort_values(ascending=False)[:10].index )]

4536    1.000000
4243    0.976263
2102    0.969931
3698    0.963239
2543    0.961248
3933    0.958763
3798    0.957797
8905    0.956525
2762    0.954269
3100    0.954162
Name: 4536, dtype: float64


,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,elo
2101,2102,6597651,6597651,6791425,63,1597801577,9.781598e+12,Paolo Bacigalupi,2009.0,The Windup Girl,...,50059,5473,1859,4274,11680,19411,12835,https://images.gr-assets.com/books/1278940608m...,https://images.gr-assets.com/books/1278940608s...,1200.0
2542,2543,7141642,7141642,7405757,67,1400065453,9.781400e+12,David Mitchell,2010.0,The Thousand Autumns of Jacob de Zoet,...,42312,5175,859,2105,7073,17293,14982,https://images.gr-assets.com/books/1320540908m...,https://images.gr-assets.com/books/1320540908s...,1200.0
2761,2762,3819326,3819326,3863622,38,425226921,9.780425e+12,Nalini Singh,2009.0,NaN,...,58508,3599,1565,2646,9498,18531,26268,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,1200.0
3099,3100,322351,322351,816361,45,440421853,9.780440e+12,Thomas Rockwell,1973.0,How to Eat Fried Worms,...,39209,853,1070,3108,11838,11890,11303,https://images.gr-assets.com/books/1320519921m...,https://images.gr-assets.com/books/1320519921s...,1200.0
3697,3698,1001896,1001896,818039,56,590225170,9.780590e+12,Blanche Fisher Wright,1916.0,The Real Mother Goose,...,32047,255,275,829,4913,8055,17975,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,1200.0
3797,3798,11100431,11100431,16022532,19,316187712,9.780316e+12,Michael J. Sullivan,2012.0,Heir of Novron,...,28151,1507,119,344,2314,8956,16418,https://images.gr-assets.com/books/1307960132m...,https://images.gr-assets.com/books/1307960132s...,1200.0
3932,3933,14990,14990,2398507,34,393329127,9.780393e+12,Mary Roach,2005.0,Spook: Science Tackles the Afterlife,...,27737,2306,802,3023,9144,9144,5624,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,1200.0
4242,4243,12326644,12326644,17305288,47,145162607X,9.781452e+12,"Philippa Gregory, Mirjana Valent",2012.0,The Kingmaker's Daughter,...,28940,2427,207,1090,6870,12606,8167,https://images.gr-assets.com/books/1332688725m...,https://images.gr-assets.com/books/1332688725s...,1200.0
4535,4536,211099,211099,204315,49,812932293,9.780813e+12,Richard Branson,2011.0,"Losing My Virginity: How I've Survived, Had Fu...",...,28791,844,1260,1359,5181,10085,10906,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,1200.0
8904,8905,7897620,7897620,11141856,36,1400065933,9.781400e+12,Fannie Flagg,2010.0,I Still Dream About You,...,13971,2183,377,1490,4815,4671,2618,https://images.gr-assets.com/books/1320471793m...,https://images.gr-assets.com/books/1320471793s...,1200.0
